In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import pysindy as ps
import matplotlib.pyplot as plt
import warnings
from imp import reload

from utilities import *
from bootstrapping import *
from optimizers import *
from weak_forms import *

In [2]:
def run_search(Theta, X_dot, var, 
               n_bootstraps = 100, n_features_to_drop = 1, n_max_features = 5,
               feature_names_list = [" "], print_hierarchy = 0):
    """
    Performs a searcg of the ALASSO solution (regularization) path, indentifies supports and fits them with OLS, returning the optimal model
    Can bootstrap data (n_bootstraps) and features (n_features_to_drop) randomly in the feature selection (support finding) process. 
    Only keeps supports with a number of features below n_max_features 
    IN: Theta [n_points, n_features], X_dot [n_points], supports [n_supports, n_features]

    """
    coefs = np.zeros((Theta.shape[1], 100, n_bootstraps))
    sup = np.zeros_like(coefs[:,:,0])
    supports = []
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        for i in range(n_bootstraps):
            if n_bootstraps != 1:
                Theta_new, X_dot_new, inds = bootstrapping(Theta, X_dot, n_features_to_drop = n_features_to_drop)
            else:
                Theta_new, X_dot_new, inds = Theta, X_dot, np.arange(Theta.shape[1])
            alphas, coefs[inds, :, i] = ALASSO_path(Theta_new, X_dot_new[:,var].reshape(-1,1),)
            supports += identify_unique_supports(coefs[:,:,i], n_max_features = n_max_features)
            sup += np.abs(coefs[:,:,i]) > 0
        
    supports = remove_duplicates(supports)
    coef_list, score, n_terms = fit_supports(Theta, X_dot[:,var], supports)
    opt_coefs, index_min = find_optimal_support(coef_list, score, n_terms)
    print_hierarchy_f(print_hierarchy, coef_list, n_terms, score, feature_names_list)
    return opt_coefs, score[index_min]

## 1. Test differential algorithm in Lotka-Volterra

In [33]:
def generate_Theta(X):
    Theta = np.vstack((np.ones_like(X[:,0]),
                    X[:,0], X[:,1], X[:,2], 
                    X[:,0]*X[:,0], X[:,1]*X[:,1], X[:,1]*X[:,2],
                    X[:,0]*X[:,1], X[:,0]*X[:,2], X[:,1]*X[:,2])).T
    return Theta

X, t = generate_data(10, 10/0.002, "lorenz")
X_dot = ps.SmoothedFiniteDifference()(X, t)
Theta = generate_Theta(X)

feature_name_list = ["1", "x", "y", "z", "x^2", "y^2", "z^2", "x y", "x z", "y z"]
var_list = ["x", "y", "z"]

opt_coefs = np.zeros((X.shape[1], Theta.shape[1]))

print("Denoised: Without ensembling")
for var in range(X.shape[1]):
    opt_coefs[var], score = run_search(Theta, X_dot, var,
                            n_bootstraps=1, n_features_to_drop=0,
                            feature_names_list=feature_name_list, print_hierarchy=0)
    print_model(opt_coefs[var], score, feature_name_list, var_list[var])

###################
X = add_noise(X, 0.05)
X_dot = ps.SmoothedFiniteDifference()(X, t)
Theta = generate_Theta(X)

print("\n5% Noise: Without ensembling")
for var in range(X.shape[1]):
    opt_coefs[var], score = run_search(Theta, X_dot, var,
                            n_bootstraps=1, n_features_to_drop=0,
                            feature_names_list=feature_name_list, print_hierarchy=0)
    print_model(opt_coefs[var], score, feature_name_list, var_list[var])

print("\n5% Noise: With ensembling")
for var in range(X.shape[1]):
    opt_coefs[var], score = run_search(Theta, X_dot, var,
                            n_bootstraps=50, n_features_to_drop=1,
                            feature_names_list=feature_name_list, print_hierarchy=0)
    print_model(opt_coefs[var], score, feature_name_list, var_list[var])

Denoised: Without ensembling
[1.00] x_dot = +   -10.00 x +    10.00 y 
[1.00] y_dot = +    25.54 x +    -0.95 x z 
[1.00] z_dot = +    -2.67 z +     1.00 x y 

5% Noise: Without ensembling
[-1.43] x_dot = +    -6.93 x +     8.86 y +    -0.06 x z 
[-0.03] y_dot = +    25.16 x +    -0.94 x z 
[0.27] z_dot = +    -2.63 z +     0.98 x y 

5% Noise: With ensembling
[-1.44] x_dot = +    -9.41 x +     9.50 y 
[-0.03] y_dot = +    25.16 x +    -0.94 x z 
[0.27] z_dot = +    -2.63 z +     0.98 x y 


## 2. Test weak form algorithm in Lotka-Volterra

In [32]:
X, t = generate_data(10, 10/0.002, "lorenz")
Theta = generate_Theta(X)
Theta_sm, X_sm = compute_weak_forms(Theta, X, t)

feature_name_list = ["1", "x", "y", "z", "x^2", "y^2", "z^2", "x y", "x z", "y z"]
var_list = ["x", "y", "z"]

opt_coefs = np.zeros((X.shape[1], Theta.shape[1]))

print("Denoised: Weak without ensembling")
for var in range(X.shape[1]):
    opt_coefs[var], score = run_search(Theta_sm, X_sm, var,
                            n_bootstraps=1, n_features_to_drop=0,
                            feature_names_list=feature_name_list, print_hierarchy=0)
    print_model(opt_coefs[var], score, feature_name_list, var_list[var])

###################
X = add_noise(X, 0.15)
Theta = generate_Theta(X)
Theta_sm, X_sm = compute_weak_forms(Theta, X, t)

print("\n15% Noise: Weak without ensembling")
for var in range(X.shape[1]):
    opt_coefs[var], score = run_search(Theta_sm, X_sm, var,
                            n_bootstraps=1, n_features_to_drop=0,
                            feature_names_list=feature_name_list, print_hierarchy=0)
    print_model(opt_coefs[var], score, feature_name_list, var_list[var])

print("\n15% Noise: Weak with ensembling")
for var in range(X.shape[1]):
    opt_coefs[var], score = run_search(Theta_sm, X_sm, var,
                            n_bootstraps=50, n_features_to_drop=1,
                            feature_names_list=feature_name_list, print_hierarchy=0)
    print_model(opt_coefs[var], score, feature_name_list, var_list[var])

Denoised: Weak without ensembling
[1.00] x_dot = +   -10.00 x +    10.00 y 
[1.00] y_dot = +    24.73 x +    -0.92 x z 
[1.00] z_dot = +    -2.67 z +     1.00 x y 

15% Noise: Weak without ensembling
[0.91] x_dot = +    -0.08 1 +   -10.15 x +     9.97 y 
[0.94] y_dot = +    24.90 x +     0.00 z +    -0.92 x z 
[0.93] z_dot = +     0.04 x +    -2.70 z +     1.03 x y 

15% Noise: Weak with ensembling
[0.91] x_dot = +   -10.44 x +    10.27 y 
[0.94] y_dot = +    24.72 x +    -0.91 x z 
[0.93] z_dot = +    -2.74 z +     1.04 x y 
